In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import os
import time
import requests
import warnings
warnings.filterwarnings('ignore')


In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [81]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars Page -- Latest Article

In [4]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [5]:
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [6]:
#from pprint import pprint
#pprint(soup)

In [7]:
print(soup.find('li', class_='slide').find('h3').text)
#print(soup.find('li', class_='slide', limit=5))
#print(soup.title(string=True))
#print(soup('h3'))
print(soup.find('div', class_='article_teaser_body').text)

The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. 


In [8]:
# Retrieve the latest element that contains news title and news_paragraph
news_title = soup.find('li', class_='slide').find('h3').text
#soup.find('div', class_='content_title').find('a').text #--updated code
news_p = soup.find('div', class_='article_teaser_body').text

# Display scraped data 
print(news_title)
print(news_p)

The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. 


## JPL Mars Featured Image

In [9]:
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image_url)

In [10]:
html_image = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html_image, "html.parser")

# Retrieve background-image url from style tag 
image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Website Url 
main_url = "https://www.jpl.nasa.gov"

# Concatenate website url with scraped route
image_url = main_url + image_url

# Display full link to featured image
image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17172-1920x1200.jpg'

#### Slight variation to scrape feature image

In [11]:
print(soup.article['style'])

background-image: url('/spaceimages/images/wallpaper/PIA17172-1920x1200.jpg');


In [12]:
img_text1 = soup.article['style'].replace("background-image: url('", main_url)
img_url = img_text1.replace("');", '')
print(img_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17172-1920x1200.jpg


## Mars Weather Twitter 

In [101]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
time.sleep(1)

In [102]:
html_weather = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html_weather, 'html.parser')

# Find all elements that contain tweets
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')

# Retrieve all elements that contain news title in the specified range
# Look for entries that display weather related words to exclude non weather related tweets 
for tweet in latest_tweets: 
    mars_weather = tweet.find('p').text
    if 'Sol' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

In [112]:
tweets = soup.find_all('div', 
                          class_="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
#[0].text
print(tweets)

[<div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 556 (2020-06-19) low -90.8ºC (-131.4ºF) high -6.5ºC (20.4ºF)
winds from the W at 7.3 m/s (16.3 mph) gusting to 21.3 m/s (47.6 mph)
pressure at 7.50 hPa</span></div>, <div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 555 (2020-06-18) low -91.0ºC (-131.8ºF) high -3.5ºC (25.6ºF)
winds from the W at 5.2 m/s (11.7 mph) gusting to 19.3 m/s (43.3 mph)
pressure at 7.50 hPa</span></div>, <div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 554 (2020-06-17) low -92.4ºC (-134.3ºF) high -4.0ºC 

In [119]:
latest_tweets=[]
latest_weather = tweets[0].text

for i, tweet in enumerate(tweets):
    #print(tweet.text)
    latest_tweets.append(tweet.text)
    if latest_weather != "InSight sol%":
        latest_weather = tweets[i].text

In [120]:
print(latest_weather)

InSight sol 550 (2020-06-13) low -92.1ºC (-133.7ºF) high 0.9ºC (33.7ºF)
winds from the SW at 4.8 m/s (10.6 mph) gusting to 20.7 m/s (46.3 mph)
pressure at 7.50 hPa


## Mars Facts WebScraping

In [122]:
url = "https://space-facts.com/mars/"
browser.visit(url)
# Use Pandas to "read_html" to parse the URL


In [124]:
tables = pd.read_html(url)
#Find Mars Facts DataFrame in the lists of DataFrames
df = tables[1]


In [125]:
df

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-153 to 20 °C,-88 to 58°C


In [126]:
#Assign the columns
df.columns = ['Description', 'Value']
df

ValueError: Length mismatch: Expected axis has 3 elements, new values have 2 elements

In [ ]:

html_table = df.to_html(table_id="html_tbl_css",justify='left',index=False)
data = df.to_dict(orient='records')  # Here's our added param..
df



In [17]:

#Save html to folder and show as html table string
mars_df = df.to_html(classes = 'table table-striped')
print(mars_df)

NameError: name 'df' is not defined

In [18]:
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)

NameError: name 'browser' is not defined

In [ ]:
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = bs( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls